In [7]:
"""
This notebook is the result of playing around with a logpolar distance metric 
(check out http://www.cse.cuhk.edu.hk/~ttwong/papers/asciiart/asciiart.pdf)

In the end, the weighted moving SSIM gave better results, so I dropped this.
"""

import numpy as np
import matplotlib.pyplot as plt

from skimage import data
from skimage.transform import warp_polar, rotate, rescale
from skimage.util import img_as_float

import matplotlib.pyplot as plt

import cv2

from glob import glob

from sys import getsizeof as sz

In [8]:
def get_tileset(fp):
    with open(fp, 'rb') as _if:
        n_tiles = ord(_if.read(1))
        n_rows = ord(_if.read(1))
        n_cols = ord(_if.read(1))
        
        n_pixels = n_rows * n_cols
        
        tileset = []
        
        for i in range(n_tiles):
            tile = np.frombuffer(_if.read(n_pixels), dtype = np.ubyte)    
            tileset.append(tile)
        
        tileset = np.asarray(tileset)
        
        return n_rows, n_cols, tileset
    
def get_tileset_dir(path ):
    
    tileset = []
    for fp in glob(path + "/*"):
        img = cv2.imread(fp)
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        if img is not None:
            tileset.append(img)
    
    return np.asarray(tileset)

In [9]:
tileset = get_tileset_dir("./tilesets/courit_12")

n_rows = tileset[0].shape[0]
n_cols = tileset[0].shape[1]

In [10]:
padded = [x.reshape((n_rows,n_cols)) for x in tileset]
padded = [np.pad(x, 5, 'maximum') for x in padded]

radius = int(np.sqrt((n_cols/2)**2 + (n_rows/2)**2))+1

In [40]:
def create_mask(seq, shape):
    mask = []
    
    for num in seq:
        mask.append(np.ones(shape) * num)
    
    mask = np.hstack(mask)
    
    return mask

mask = create_mask([1, 0.9, 1, 0.9, 0.8, 0.9, 1, 0.9, 1], (12, 5))

In [41]:
polar = []

base_x = 5 + n_cols/2
base_y = 5 + n_rows/2 + 1

for ind, char in enumerate(padded):
    polar.append([])
    
    for h in range(-1,2):
        for v in range(-1,2):
            
            warped = cv2.warpPolar(char, (5, 12), (base_x + h*2, base_y + v*2), radius, flags=cv2.WARP_POLAR_LOG)
    
            polar[ind].append(warped)    

polar = [np.asarray(char) for char in polar]

#polar = [[cv2.GaussianBlur(_slice, (3,3), 0) for _slice in char] for char in polar]

polar = [np.hstack(char) for char in polar]

polar = np.asarray(polar)

In [43]:
def get_empty_ind(tileset):
    for ind, tile in enumerate(tileset):
        if (tile == 255).all():
            return ind

def polar_difference(patch1, patch2):
    
    mean1 = np.mean(patch1)
    mean2 = np.mean(patch2)
    
    diff = np.sum(np.absolute(patch1 - patch2)*mask) / (mean1 + mean2)
    
    return diff

def best_fit_index(patch, patches, empty_ind=None):
    best_diff = polar_difference(patch, patches[0])
    best_ind = 0
    
    for ind in range(1, len(patches)):
        diff = polar_difference(patch, patches[ind])
        
        if empty_ind == ind:
            diff *= 1.1
            
        if diff < best_diff:
            best_diff = diff
            best_ind = ind

    return best_ind

def get_logpolar_transform(img, radius, n_rows, n_cols):
    polar_img = []

    for y in range(radius, img.shape[0]-radius, n_rows):

        polar_img.append([])
        for x in range(radius, img.shape[1]-radius, n_cols):

            polar_img[-1].append([])

            for h in range(-1,2):
                for v in range(-1,2):
                    polarized = cv2.warpPolar(img, (5,12), (x + n_cols/2 + h*2, y + n_rows/2 + 1 + v*2), radius, flags=cv2.WARP_POLAR_LOG)

                    polar_img[-1][-1].append(polarized)


            
            #polar_img[-1][-1] = [[cv2.GaussianBlur(_slice, (3,3), 0) for _slice in patch] for patch in polar_img[-1][-1] ]           
            polar_img[-1][-1] = np.hstack(polar_img[-1][-1])
            polar_img[-1][-1] = np.squeeze(polar_img[-1][-1])
            #polar_img[-1][-1] = cv2.GaussianBlur(polar_img[-1][-1], (3,3), 0)
            

        polar_img[-1] = np.asarray(polar_img[-1])


    polar_img = np.asarray(polar_img) 

    return polar_img

def get_closest_ascii_img(img, tileset, polar_tileset, n_rows, n_cols, radius):
    
    ascii_img = []
    
    polar_img = get_logpolar_transform(img, radius, n_rows, n_cols)
    
    empty_ind = get_empty_ind(tileset)
    
    for row in polar_img:
        
        ascii_img.append([])
        
        for polar_patch in row:
            
            ind = best_fit_index(polar_patch, polar_tileset, empty_ind)
            
            ascii_img[-1].append(tileset[ind].reshape(n_rows, n_cols))
            
        ascii_img[-1] = np.asarray(ascii_img[-1])
        ascii_img[-1] = np.hstack(ascii_img[-1])
        
    
    ascii_img = np.asarray(ascii_img)
    ascii_img = np.vstack(ascii_img)
    
    return ascii_img   